In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-11-13 17:05:38.399552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
feature_arr = np.load("./cleaned_data/feature_arr_ids.npy")
target_arr = np.load("./cleaned_data/target_arr_ids.npy")

np.nan_to_num(feature_arr, copy=False, nan=0)
np.nan_to_num(target_arr, copy=False, nan=0)
print('done')


done


In [5]:
#df = pd.DataFrame(feature_arr.reshape(feature_arr.shape[0], -1)).sort_values([0,1,2])

In [8]:
reshaped_x = feature_arr.reshape(feature_arr.shape[0], -1)

#print(f"Shape of dataframe = {df.shape}")
#print(f"70% of length = {df.shape[0]*.7}")
#print(f"15% of length = {df.shape[0]*.15}")

# training set will start at. Pick these values so at the start of a new play
train_index_end = 196648
val_index_end = 238779

x_preprocess_train = reshaped_x[0:train_index_end, :].reshape(-1, feature_arr.shape[1], feature_arr.shape[2])
x_preprocess_val = reshaped_x[train_index_end:val_index_end, :].reshape(-1, feature_arr.shape[1], feature_arr.shape[2])
x_preprocess_test = reshaped_x[val_index_end:, :].reshape(-1, feature_arr.shape[1], feature_arr.shape[2])

y_train = target_arr[0:train_index_end]
y_val = target_arr[train_index_end:val_index_end]
y_test = target_arr[val_index_end:]



#df.iloc[train_index_start+42130:train_index_start+42142+10, 0:5]

In [19]:
print(x_preprocess_train[0:2])

[[[ 2.0210909e+09  9.7000000e+01  6.0000000e+00  2.5511000e+04
    2.0000000e+00  1.0405000e+02  6.7500000e+00  2.2300000e+00
    1.9200000e+00  1.1560000e+02  7.0440000e+01]
  [ 2.0210909e+09  9.7000000e+01  6.0000000e+00  3.5481000e+04
    1.0000000e+00  1.0638000e+02  1.6300000e+00  6.8000000e-01
    1.7100000e+00  2.7631000e+02  2.4479000e+02]
  [ 2.0210909e+09  9.7000000e+01  6.0000000e+00  3.5634000e+04
    1.0000000e+00  8.6200000e+01  1.0790000e+01  6.3000000e+00
    4.8400000e+00  1.0943000e+02  4.6550000e+01]
  [ 2.0210909e+09  9.7000000e+01  6.0000000e+00  3.9985000e+04
    2.0000000e+00  9.5800000e+01  2.1040000e+01  6.3500000e+00
    2.0900000e+00  1.2946000e+02  7.6550000e+01]
  [ 2.0210909e+09  9.7000000e+01  6.0000000e+00  4.0151000e+04
    2.0000000e+00  1.0645000e+02  5.9400000e+00  2.2100000e+00
    9.0000000e-01  1.7527000e+02  1.3222000e+02]
  [ 2.0210909e+09  9.7000000e+01  6.0000000e+00  4.1233000e+04
    2.0000000e+00  8.7600000e+01 -1.4220000e+01  8.0300000e+00

In [9]:
#x_preprocess_train, x_preprocess_test, y_train, y_test = train_test_split(feature_arr, target_arr, test_size=0.3, random_state=5)
#x_preprocess_test, x_preprocess_val, y_test, y_val = train_test_split(x_preprocess_test, y_test, test_size=0.5, random_state=5)

In [20]:

x_train = x_preprocess_train[:,:,4:11]
x_val = x_preprocess_val[:,:,4:11]
x_test = x_preprocess_test[:,:,4:11]

np.random.shuffle(x_train)
np.random.shuffle(x_val)
np.random.shuffle(x_test)

In [22]:
''' 
Feature list:
['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir']
'''

" \nFeature list:\n['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir']\n"

In [23]:

# standardize data
train_mu = np.mean(x_train.reshape(-1,x_train.shape[-1]),axis=0)
train_std = np.std(x_train.reshape(-1,x_train.shape[-1]),axis=0)

x_train[:,:,1:] = (x_train[:,:,1:] - train_mu[1:])/train_std[1:]
x_val[:,:,1:] = (x_val[:,:,1:] - train_mu[1:])/train_std[1:]
x_test[:,:,1:] = (x_test[:,:,1:] - train_mu[1:])/train_std[1:]


In [24]:
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
# test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

print ("number of training examples = " + str(x_train.shape[0]))
print ("number of test examples = " + str(x_test.shape[0]))
print ("X_train shape: " + str(x_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_val shape: " + str(x_val.shape))
print ("Y_val shape: " + str(y_val.shape))
print ("X_test shape: " + str(x_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 196648
number of test examples = 42170
X_train shape: (196648, 23, 7)
Y_train shape: (196648, 2)
X_val shape: (42131, 23, 7)
Y_val shape: (42131, 2)
X_test shape: (42170, 23, 7)
Y_test shape: (42170, 2)


In [25]:
import keras.backend as K

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    mse = tf.keras.losses.MeanSquaredError()
    return bce(y_true[:,0], y_pred[:,0]) + mse(y_true[:,1], y_pred[:,1])

def bce_metric(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true[:,0], y_pred[:,0], from_logits=True))
    
    #logit_max = K.clip(y_pred, 1e-12, 1e12)
    #tf.print(y_pred[:,0])
    #tf.print(logit_max[:,0])
    #probs = 1/(1+K.exp(-logit_max[:,0]))
    #return -K.mean( y_true[:,0]*K.log(probs) + (1-y_true[:,0])*K.log(1-probs))

def mse_metric(y_true, y_pred):
    return K.mean(K.square(y_pred[:,1] - y_true[:,1]), axis=-1)

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true[:,0] * y_pred[:,0], 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,0], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true[:,0] * y_pred[:,0], 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred[:,0], 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision



In [26]:
model = tf.keras.Sequential([
            tfl.Flatten(input_shape=(23, 7)),
            tfl.Dense(100, activation='relu'),
            tfl.Dense(80, activation='relu'), 
            tfl.Dense(2,activation=None)
        ])
model.compile(optimizer='adam',
              loss=my_loss,
              metrics=[bce_metric, mse_metric, recall, precision])

In [27]:
NUM_EPOCHS = 5
history = model.fit(x_train, y_train, epochs=NUM_EPOCHS)

Epoch 1/5
6146/6146 [==============================] - 13s 2ms/step - loss: 1.3716 - bce_metric: 0.2849 - mse_metric: 1.0865 - recall: 1.1390e-04 - precision: 3.1960e-05
Epoch 2/5
6146/6146 [==============================] - 12s 2ms/step - loss: 1.3586 - bce_metric: 0.2828 - mse_metric: 1.0760 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 3/5
6146/6146 [==============================] - 12s 2ms/step - loss: 1.3571 - bce_metric: 0.2825 - mse_metric: 1.0745 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 4/5
6146/6146 [==============================] - 12s 2ms/step - loss: 1.3553 - bce_metric: 0.2819 - mse_metric: 1.0733 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 5/5
6146/6146 [==============================] - 16s 3ms/step - loss: 1.3527 - bce_metric: 0.2810 - mse_metric: 1.0716 - recall: 0.0000e+00 - precision: 0.0000e+00


In [29]:
val_loss, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val, y_val, verbose=2)

metrics_df = pd.DataFrame(history.history)


1317/1317 - 2s - loss: 2.0911 - bce_metric: 0.8982 - mse_metric: 1.1923 - recall: 0.0036 - precision: 0.0110 - 2s/epoch - 1ms/step


In [ ]:
metrics_df

,loss,bce_metric,mse_metric,recall,precision
0,1.156930,0.231311,0.925618,0.085188,0.186666
1,0.696731,0.137030,0.559678,0.418932,0.665055
2,0.475289,0.090597,0.384704,0.606448,0.788535
3,0.357305,0.064594,0.292707,0.713964,0.846681
4,0.288632,0.049381,0.239251,0.765980,0.866968
5,0.242342,0.038002,0.204356,0.809271,0.886129
6,0.201743,0.029613,0.172126,0.836978,0.896455
7,0.182937,0.024706,0.158225,0.857228,0.903895
8,0.158901,0.019616,0.139280,0.873908,0.908965
9,0.142864,0.016687,0.126193,0.888765,0.918984


In [ ]:
print(f"val loss = {val_loss}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 0.20938993990421295
val_bce = 0.02983475849032402
val_mse = 0.17955544590950012
val_recall = 0.8652931451797485
val_precision = 0.8892954587936401


In [ ]:
model_string = f"models/basic_model/epochs{NUM_EPOCHS}.ckpt"
model.save_weights(model_string.format(epoch=0))

In [ ]:
# # Define baseline model
# model = tf.keras.Sequential([
#             tfl.Flatten(input_shape=(23, 7)),
#             tfl.Dense(100, activation='relu'),
#             tfl.Dense(80, activation='relu'), 
#             tfl.Dense(2,activation=None)
#         ])
# model.compile(optimizer='adam',
#               loss='mean_squared_error',
#               metrics=[tf.keras.metrics.MeanSquaredError()])
# history = model.fit(x_train, y_train, epochs=3)
# val_loss, val_mse = model.evaluate(x_val, y_val, verbose=2)
# print('Val mse:', val_mse)
# metrics_df = pd.DataFrame(history.history)
# #metrics_df.loss.plot(title='Model loss',figsize=(6,3)).set(xlabel='Epoch',ylabel='Loss')